In [1]:
pip install  numpy pandas

In [3]:
import numpy as np
import pandas as pd
import random

# 配置參數
NUM_STOCKS = 10  # 總股票數量
POPULATION_SIZE = 50  # 初始族群大小
GENERATIONS = 100  # 最大迭代次數
MUTATION_RATE = 0.1  # 突變率

# 模擬股票回報率和風險
def simulate_stock_data(num_stocks):
    np.random.seed(42)
    returns = np.random.uniform(0.05, 0.2, num_stocks)  # 模擬每支股票的回報率
    risks = np.random.uniform(0.1, 0.5, num_stocks)  # 模擬每支股票的風險
    return pd.DataFrame({"Stock": [f"Stock_{i+1}" for i in range(num_stocks)], "Return": returns, "Risk": risks})

# 定義適應函數
# 目標是最大化回報率並最小化風險
def fitness_function(portfolio, stock_data):
    total_return = np.sum(portfolio * stock_data["Return"].values)
    total_risk = np.sqrt(np.sum((portfolio * stock_data["Risk"].values) ** 2))
    return total_return - total_risk  # 簡單的回報與風險權衡模型

# 初始化族群
def initialize_population(pop_size, num_stocks):
    return np.random.randint(2, size=(pop_size, num_stocks))


# 選擇
# 根據適應值選擇最優染色體
def select_parents(population, fitness_scores):
    # 確保適應值為正數
    min_fitness = np.min(fitness_scores)
    if min_fitness < 0:
        fitness_scores += abs(min_fitness) + 1e-6  # 平移所有適應值使其為正數

    # 防止分母為零
    total_fitness = np.sum(fitness_scores)
    if total_fitness == 0:
        probabilities = np.ones_like(fitness_scores) / len(fitness_scores)  # 平均分佈
    else:
        probabilities = fitness_scores / total_fitness

    # 選擇父母染色體
    parents_indices = np.random.choice(len(population), size=2, replace=False, p=probabilities)
    return population[parents_indices]


# 交配（單點交叉）
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# 突變
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # 基因翻轉
    return chromosome

# 運行基因演算法
def genetic_algorithm(stock_data):
    num_stocks = len(stock_data)
    population = initialize_population(POPULATION_SIZE, num_stocks)

    for generation in range(GENERATIONS):
        fitness_scores = np.array([fitness_function(ind, stock_data) for ind in population])

        new_population = []
        while len(new_population) < POPULATION_SIZE:
            parents = select_parents(population, fitness_scores)
            child1, child2 = crossover(parents[0], parents[1])
            new_population.append(mutate(child1, MUTATION_RATE))
            new_population.append(mutate(child2, MUTATION_RATE))

        population = np.array(new_population[:POPULATION_SIZE])

        # 輸出每代最佳適應值
        best_fitness = np.max(fitness_scores)
        print(f"Generation {generation+1}: Best Fitness = {best_fitness:.4f}")

    # 返回最佳染色體及其適應值
    best_index = np.argmax(fitness_scores)
    best_chromosome = population[best_index]
    best_return = np.sum(best_chromosome * stock_data["Return"].values)
    best_risk = np.sqrt(np.sum((best_chromosome * stock_data["Risk"].values) ** 2))
    return best_chromosome, best_return, best_risk

# 主程式執行邏輯
def main():
    stock_data = simulate_stock_data(NUM_STOCKS)
    print("模擬股票數據:")
    print(stock_data)

    print("\n運行基因演算法...")
    best_portfolio, best_return, best_risk = genetic_algorithm(stock_data)

    print("\n最佳投資組合:")
    for i, selected in enumerate(best_portfolio):
        if selected:
            print(f"{stock_data['Stock'][i]}: Included")
    print(f"\n投資組合回報率: {best_return:.4f}")
    print(f"投資組合風險: {best_risk:.4f}")

if __name__ == "__main__":
    main()

模擬股票數據:
      Stock    Return      Risk
0   Stock_1  0.106181  0.108234
1   Stock_2  0.192607  0.487964
2   Stock_3  0.159799  0.432977
3   Stock_4  0.139799  0.184936
4   Stock_5  0.073403  0.172730
5   Stock_6  0.073399  0.173362
6   Stock_7  0.058713  0.221697
7   Stock_8  0.179926  0.309903
8   Stock_9  0.140167  0.272778
9  Stock_10  0.156211  0.216492

運行基因演算法...
Generation 1: Best Fitness = 0.4472
Generation 2: Best Fitness = 0.4112
Generation 3: Best Fitness = 0.3673
Generation 4: Best Fitness = 0.4969
Generation 5: Best Fitness = 0.4511
Generation 6: Best Fitness = 0.5514
Generation 7: Best Fitness = 0.5489
Generation 8: Best Fitness = 0.4477
Generation 9: Best Fitness = 0.4977
Generation 10: Best Fitness = 0.4835
Generation 11: Best Fitness = 0.3972
Generation 12: Best Fitness = 0.4597
Generation 13: Best Fitness = 0.4034
Generation 14: Best Fitness = 0.5267
Generation 15: Best Fitness = 0.5260
Generation 16: Best Fitness = 0.3976
Generation 17: Best Fitness = 0.4515
Generati